## Accessing USDA Food Data Central API for Nutritional Information

This script interacts with the USDA Food Data Central API to search for food items and retrieve nutritional information.

In [4]:
#!pip install requests
import requests

# Define your API key (it should be the actual key, not "OAS3.0:")
api_key = 'M3175TAXsEcfdKkCOpNO9VcbwjQtFOkl9rUDAiaP'

# Function to search for a food item and get its FDC ID
def search_food(query):
    #This is the data set for the USDA government
    url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    # my parameters are query for the 
    params = {
        "query": query, # search term enterd by user 
        "api_key": api_key, 
        "pageSize": 1  # Return only the top result
    }
    # using request.get() to send a get request to the API endpoint
    response = requests.get(url, params=params)
    #lines checks for the Https status and if it 200 then its okay
    if response.status_code == 200:
        # returns the json format to dictionary
        data = response.json()
        #checks for food items 
        if data['foods']:
            # this will get the first result 
            food_item = data['foods'][0]
            # return the food id and its description
            return food_item['fdcId'], food_item['description']
        else:
           #No foods were found
            print("No foods found.")
            return None, None
    else:
      # it will print out the error responce code
        print(f"Error: {response.status_code}")
        return None, None

# Function to retrieve detailed nutritional information using FDC ID
def get_food_details(fdc_id):

    url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
    params = {
        "api_key": api_key,
        "format": "full"#All available details for food
    }
    # Returns the full detail information for the food 
    response = requests.get(url, params=params)
    # Same thing as earlier 
    if response.status_code == 200:
        food_details = response.json()
        return food_details
    else:
        print(f"Error: {response.status_code}")
        return None

# whatever the user wants

foodname = str(input("Please enter the name of the object you want the nutrition data for: "))

# Example usage
if __name__ == "__main__":
    food_query = foodname
    fdc_id, description = search_food(food_query)
    
    if fdc_id:
        print(f"FDC ID: {fdc_id}, Description: {description}")
        food_details = get_food_details(fdc_id)
        
        # Print out some of the nutritional information
        if food_details:
            print(f"\nNutritional Information for {description}:")
            for nutrient in food_details.get('foodNutrients', []):
                print(f"{nutrient['nutrient']['name']}: {nutrient['amount']} {nutrient['nutrient']['unitName']}")
    else:
        print("Could not retrieve food details.")

FDC ID: 770088, Description: OREO COOKIES

Nutritional Information for OREO COOKIES:
Carbohydrate, by difference: 73.0 g
Total lipid (fat): 20.0 g
Fiber, total dietary: 2.5 g
Sodium, Na: 400.0 mg
Total Sugars: 38.0 g
Protein: 5.0 g
Fatty acids, total saturated: 9.8 g


## Extract Video Frames in Smaller Batches

1. Process multiple videos from the input folder.
2. Create a new numbered subfolder within the output folder for each video.
3. Name each subfolder with a sequential number followed by _vid.
4. Store the extracted frames as JPG files within their respective subfolders.

In [1]:
import cv2
import os

def video_to_frames_in_smaller_batches(video_folder, output_folder, desired_fps):
    print(f"Processing videos in {video_folder} to {output_folder} with {desired_fps} FPS")
    # List all video files in the selected folder
    video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f)) and f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    if not video_files:
        print("No video files found in the selected folder.")
        return

    for idx, video_file in enumerate(video_files, start=1):
        video_path = os.path.join(video_folder, video_file)

        # Create a new subfolder in the output folder named <number>_vid
        video_output_folder = os.path.join(output_folder, f'{idx}_vid')
        os.makedirs(video_output_folder, exist_ok=True)

        # Open the video file
        cap = cv2.VideoCapture(video_path)
        original_fps = cap.get(cv2.CAP_PROP_FPS)

        if original_fps <= 0:
            print(f"Warning: Unable to retrieve FPS for video {video_file}. Skipping...")
            continue

        # Calculate the frame interval to match the desired FPS
        frame_interval = int(round(original_fps / desired_fps))

        if frame_interval <= 0:
            frame_interval = 1  # Ensure at least every frame is processed

        frame_count = 0
        saved_frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Save frames at the specified interval into the specific subfolder
            if frame_count % frame_interval == 0:
                frame_filename = os.path.join(video_output_folder, f'frame_{saved_frame_count:05d}.jpg')
                cv2.imwrite(frame_filename, frame)
                saved_frame_count += 1

            frame_count += 1

        cap.release()
        print(f"Extracted frames from '{video_file}' into the folder '{video_output_folder}'.")

    print("Processing complete.")






## Video file selection

This code cell creates a graphical user interface (GUI) using Python's `tkinter` library to assist users in processing video files. The GUI guides the user through the following steps:

1. **Greeting the User**: Asks if they would like to process videos.
2. **Selecting a Video Folder**: Opens a dialog for the user to choose a folder containing video files.
3. **Handling Missing Files**: Provides help if the user cannot find the desired video folder.
4. **Prompting for Frames Per Second (FPS)**: Asks the user to input the desired frames per second (FPS) for the video processing.
5. **Selecting an Output Folder**: Prompts the user to choose or create a folder where the processed frames will be saved.
6. **Handling Default Output Folder**: If the user does not select an output folder, the program creates a default folder named `output_file` in the current working directory.
7. **Handling Duplicate Output Folders**: If a folder named `output_file` already exists, the program notifies the user and terminates to avoid conflicts.
8. **Processing the Videos**: Processes the videos by extracting frames at the specified FPS and saves them in the output folder.



In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog


# Function to handle when the user can't find their file (to be implemented)
def handle_cant_find_file():
    # Placeholder for the file search assistance logic
    messagebox.showinfo("Help", "Please check your folder structure and try again.")



def greet_and_prompt():
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Greet the user
    user_response = messagebox.askquestion("Welcome", "Hello! Would you like to process videos?")

    if user_response == 'yes':
        while True:
            # Ask the user to select the folder containing video files
            video_folder = filedialog.askdirectory(title="Select the Folder Containing Videos")

            if video_folder:
                break  # Break the loop if a folder is selected
            else:
                # Option for the user if they can't find the folder
                cant_find_response = messagebox.askquestion("Can't Find Folder",
                                                            "Can't find your folder? Would you like some help?",
                                                            icon='warning')
                if cant_find_response == 'yes':
                    handle_cant_find_file()
                else:
                    messagebox.showinfo("Goodbye", "You chose not to proceed. Goodbye!")
                    root.update()  # Process any pending events
                    root.quit()  # Exit the main loop
                    root.destroy()  # Destroy the Tkinter root window
                    return  # Exit the function and stop the script

        # Prompt the user to select the output folder
        select_output_response = messagebox.askquestion("Output Folder", "Would you like to select an output folder now?")
        
        if select_output_response == 'yes':
            while True:
                output_folder = filedialog.askdirectory(title="Select Output Folder")

                if output_folder:
                    messagebox.showinfo("Folder Selected", f"Output folder selected: {output_folder}")
                    break  # Proceed if a valid output folder is selected
                else:
                    # Notify the user to select a folder or quit the process
                    retry_response = messagebox.askquestion("No Folder Selected",
                                                            "You haven't selected an output folder. Would you like to try again?",
                                                            icon='warning')
                    if retry_response == 'no':
                        messagebox.showinfo("Goodbye", "You chose not to proceed. Goodbye!")
                        root.update()  # Process any pending events
                        root.quit()  # Exit the main loop
                        root.destroy()  # Destroy the Tkinter root window
                        return  # Exit the function and stop the script
        else:
            messagebox.showinfo("Goodbye", "You chose not to select an output folder. Goodbye!")
            root.update()  # Process any pending events
            root.quit()  # Exit the main loop
            root.destroy()  # Destroy the Tkinter root window
            return  # Exit the function and stop the script

        # Ask the user to input the frames per second (FPS)
        fps = simpledialog.askinteger("Frames Per Second", "Please enter the frames per second (FPS) value:", minvalue=1)

        if fps:
            # Call the function with the gathered information
            video_to_frames_in_smaller_batches(video_folder, output_folder, int(fps))
            messagebox.showinfo("Process Complete", f"Videos processed with {fps} frames per second.")
        else:
            messagebox.showinfo("No FPS Entered", "You didn't enter a valid FPS value. Goodbye!")
    
    else:
        messagebox.showinfo("Goodbye", "You chose not to proceed. Goodbye!")
    
    root.update()  # Process any pending events
    root.quit()  # Exit the main loop
    root.destroy()  # Destroy the Tkinter root window


# Call the function
greet_and_prompt()


Processing videos in /Users/taneshafuller/Documents/ProjectRene/segment-anything-2/Video_folder to /Users/taneshafuller/Documents/ProjectRene/segment-anything-2/output with 55 FPS
Extracted frames from 'IMG_5292.mov' into the folder '/Users/taneshafuller/Documents/ProjectRene/segment-anything-2/output/1_vid'.
Extracted frames from 'IMG_5291.mov' into the folder '/Users/taneshafuller/Documents/ProjectRene/segment-anything-2/output/2_vid'.
Extracted frames from 'IMG_5284.mov' into the folder '/Users/taneshafuller/Documents/ProjectRene/segment-anything-2/output/3_vid'.
Processing complete.


## Video segmentation with SAM 2
This notebook shows how to use SAM 2 for interactive segmentation in videos. It will cover the following:

- adding clicks (or box) on a frame to get and refine _masklets_ (spatio-temporal masks)
- propagating clicks (or box) to get _masklets_ throughout the video
- segmenting and tracking multiple objects at the same time

We use the terms _segment_ or _mask_ to refer to the model prediction for an object on a single frame, and _masklet_ to refer to the spatio-temporal masks across the entire video. 

## Annotating Images with Bounding Boxes for Object Detection

This script helps annotate objects in images by drawing bounding boxes and saving them as training labels for models.
If running locally using jupyter, first install `segment-anything-2` in your environment using the [installation instructions](https://github.com/facebookresearch/segment-anything-2#installation) in the repository.


## Set env for SAM2 

In [1]:
using_colab = False
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything-2.git'

    !mkdir -p videos
    !wget -P videos https://dl.fbaipublicfiles.com/segment_anything_2/assets/bedroom.zip
    !unzip -d videos videos/bedroom.zip

    !mkdir -p ../checkpoints/
    !wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt

    

In [2]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

using device: mps

Support for MPS devices is preliminary. SAM 2 is trained with CUDA and might give numerically different outputs and sometimes degraded performance on MPS. See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion.


## Loading SAM2 Video Predictor

In [4]:
from sam2.build_sam import build_sam2_video_predictor

sam2_checkpoint = "../checkpoints/sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"

predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)

FileNotFoundError: [Errno 2] No such file or directory: '../checkpoints/sam2_hiera_large.pt'